In [128]:
import pandas as pd
from datetime import datetime
from datetime import date
#Настройка вывода
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

In [141]:
#Загрузка данных
profiles = pd.read_csv("Profiles.csv")
results = pd.read_csv("Results.csv")
fields = pd.read_csv("fields1.csv", sep=';')
#Слияние данных + удаление данных по id
data = profiles.merge(fields, on = 'id')
results = results.rename({"id_vk": "id"}, axis='columns')
data = data.merge(results, on = 'id', how = 'right').drop_duplicates(subset = ['id']).set_index('id')

In [142]:
#Предобработка
#Удаление заблокированных и забаненных пользователей
data = data.loc[data.deactivated.isnull()].drop(labels='deactivated', axis = 1)
#Удаление пользователей с закрытым профилем
data = data[~data['is_closed'].isin([True])].drop(labels='is_closed', axis = 1)
#Удаление лишних признаков
status_audio = ['status_audio_artist','status_audio_id','status_audio_owner_id','status_audio_title','status_audio_duration',
                'status_audio_url','status_audio_date','status_audio_is_hq','status_audio_main_artists','status_audio_short_videos_allowed',
                'status_audio_stories_allowed','status_audio_is_explicit','status_audio_subtitle','status_audio_lyrics_id',
                'status_audio_genre_id','status_audio_stories_cover_allowed']
for_delete = ['id_moodle','дата','Unnamed: 6', 'дата.1','дата.2','friends_online_count',
              'can_access_closed','domain','bdate','followers_count_x','false']
data = data.drop(labels = status_audio, axis = 1)
data = data.drop(labels = for_delete, axis = 1)

In [143]:
#Университет, школа, карьера
data['university'] = 0
data['school'] = 0
data['career'] = 0

university_list = ['universities_chair', 'universities_chair_name','universities_city','universities_country',
                   'universities_education_form','universities_education_status','universities_faculty',
                   'universities_faculty_name','universities_graduation','universities_id','universities_name']
data[university_list] = data[university_list].fillna('0')

school_list = ['schools_city','schools_country','schools_id','schools_name','schools_type','schools_type_str',
               'schools_year_from','schools_year_graduated','schools_year_to','schools_speciality','schools_class']
data[school_list] = data[school_list].fillna('0')

career_list = ['career_city_id','career_country_id','career_group_id','career_company',
               'career_from','career_position','career_until']
data[career_list] = data[career_list].fillna('0')



for i in university_list:
    for j in data.index:
        if data[i][j] != '0':
            data['university'][j] = 1
            
data = data.drop(labels= university_list, axis = 1)

for i in school_list:
    for j in data.index:
        if data[i][j] != '0':
            data['school'][j] = 1
            
data = data.drop(labels= school_list, axis = 1)

for i in career_list:
    for j in data.index:
        if data[i][j] != '0':
            data['career'][j] = 1
            
data = data.drop(labels= career_list, axis = 1)

<ipython-input-143-2bcd0b2b7ad2>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['university'][j] = 1
<ipython-input-143-2bcd0b2b7ad2>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['school'][j] = 1
<ipython-input-143-2bcd0b2b7ad2>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['career'][j] = 1


In [144]:
#Заполненность профиля
fullness_profile_list = ['skype','mobile_phone','maiden_name','instagram','site','quotes','relation','personal_people_main',
                         'home_town','personal_alcohol','personal_langs','personal_political','personal_religion',
                         'personal_smoking','relatives_type','relatives_id','nickname','activities',
                         'personal_inspired_by','personal_life_main','books','tv','about','movies','music','home_phone',
                         'relation_partner_first_name','relation_partner_id','relation_partner_last_name','personal_religion_id',
                         'relatives_name','games','twitter','facebook','facebook_name']
data['profile_fulness'] = 0
data['profile_fulness'] = data['profile_fulness'].astype('float64')
data[fullness_profile_list] = data[fullness_profile_list].fillna('0')

for i in fullness_profile_list:
    for j in data.index:
        if data[i][j] != '0':
            data['profile_fulness'][j] += 1
           
for i in data.index:
    data['profile_fulness'][i] = data['profile_fulness'][i]/len(fullness_profile_list)*100
            
data = data.drop(labels= fullness_profile_list, axis = 1)

<ipython-input-144-3483dbcc2c38>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['profile_fulness'][j] += 1
<ipython-input-144-3483dbcc2c38>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['profile_fulness'][i] = data['profile_fulness'][i]/len(fullness_profile_list)*100


In [145]:
#Обработка возраста
data = data.rename({"bdate_1c": "age"}, axis='columns')
data.age = data.age.fillna('0')
for i in data.index:
    if data.age[i] != '0':
        born = datetime.strptime(data.age[i][:10], '%Y-%m-%d').date()
        today = date.today()
        age = today.year - born.year - ((today.month, today.day) < (born.month, born.day))
        data.age[i] = age

data.age = pd.to_numeric(data['age'])

<ipython-input-145-386086c12e81>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.age[i] = age


In [146]:
#Факультеты(Флф?)apply, lambda
data['подразделение 1'] = data['подразделение 1'].fillna('0')
technical = ['ММФ','ФТФ','ФИТ','ИПМиКН','РФФ','ИПМКН','HITS','ФтФ']
humanitarian = ['ФП','ИЭМ','САЕ ИЧЦЭ','НЮИ', 'ФсФ', 'ЮИ','ИИК','ФИПН''ФлФ', 'ФИЯ','ФЖ','ФилФ','ФФ','ФлФ',
                'САЕ ЦИТСНУ','ФСФ','ФЛФ','Управление информационной политики','кафедра истории философии','ФИПН']
natural = ['БИ','ХФ','САЕ ИУМТ','ФФК','ГГФ','САЕ ИБМ']

for i in data.index:
    if data['подразделение 1'][i] != 0:
        if data['подразделение 1'][i] in technical: 
            data['подразделение 1'][i] = '1'
        if data['подразделение 1'][i] in humanitarian: 
            data['подразделение 1'][i] = '2'
        if data['подразделение 1'][i] in natural: 
            data['подразделение 1'][i] = '3'
        if data['подразделение 1'][i] == 'ИВО':
            data['подразделение 1'][i] = '4'
        if data['подразделение 1'][i] == 'не ТГУ':
            data['подразделение 1'][i] = '0'
data['подразделение 1'] = pd.to_numeric(data['подразделение 1'])
data = data.rename({"подразделение 1": "subdivision"}, axis='columns')

<ipython-input-146-fff6916afb6c>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['подразделение 1'][i] = '2'
<ipython-input-146-fff6916afb6c>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['подразделение 1'][i] = '1'
<ipython-input-146-fff6916afb6c>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['подразделение 1'][i] = '3'
<ipython-input-146-fff6916afb6c>:19: SettingWithCopyWarning: 
A value is trying to be set o

In [147]:
#Наличие статуса
data['status'] = data['status'].fillna('0')
for i in data.index:
    if data['status'][i] != '0':
        data['status'][i] = '1'
data['status'] = pd.to_numeric(data['status'])

<ipython-input-147-44a9234a665d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['status'][i] = '1'


In [148]:
#Города и страны, вопрос
data = data.drop(labels = ['city_id',
 'city_title',
 'country_id',
 'country_title'], axis = 1)

In [149]:
#Категоризация и заполнить пропуски
#is_closed как признак(возможно),пол, возраст(в results), pd.dummies, multibinarizer, open в %(),
#Статус, работа, образвание.
#Результаты в CATEGORIALz, crossvalscore обучающая(80%), тест(10%), валидационная(10%, GrideSearchCV, подбор параметров)
#Выделить признаки
#Пол, возраст(категариальные), наличие фото, наличие друзей,альбомы,(распределение по перецентиялм разные %)
#и другие количественные данные, 

In [150]:
data.albums_count = data.albums_count.fillna(data.albums_count.mean())
data.photos_count = data.photos_count.fillna(data.photos_count.mean())
data.friends_count = data.friends_count.fillna(data.friends_count.mean())
data.groups_count = data.groups_count.fillna(data.groups_count.mean())
data.followers_count_y = data.followers_count_y.fillna(data.followers_count_y.mean())
data.subscriptions_count = data.subscriptions_count.fillna(data.subscriptions_count.mean())
data = pd.get_dummies(data, columns=['subdivision']).drop(labels = 'subdivision_0', axis =1)

In [151]:
l1 = ['Экстраверсия',
 'Сознательность',
 'Доброжелательность',
 'Эмоциональная устойчивость',
 'Открытость опыту','false.1',
 'false.2']
l2 = ['Подавленность',
 'Тревожность',
 'Стресс',
 'false.1','false.2']
data['false.1'] = data['false.1'].fillna('1')
data['false.2'] = data['false.2'].fillna('1')
data1 = data[~data['false.1'].isin(['1'])].drop(labels=l1, axis = 1)
data2 = data[~data['false.2'].isin(['1'])].drop(labels=l2, axis = 1)

In [152]:
data1.columns

Index(['sex', 'status', 'albums_count', 'photos_count', 'friends_count',
       'groups_count', 'followers_count_y', 'subscriptions_count',
       'Подавленность', 'Тревожность', 'Стресс', 'age', 'university', 'school',
       'career', 'profile_fulness', 'subdivision_1', 'subdivision_2',
       'subdivision_3', 'subdivision_4'],
      dtype='object')

In [153]:
#1
data1['Подавленность1'] = data1['Подавленность'].apply(lambda x: 1 if x > 6 else 0)
data1['Тревожность1'] = data1['Тревожность'].apply(lambda x: 1 if x > 4 else 0)
data1['Стресс1'] = data1['Стресс'].apply(lambda x: 1 if x > 9 else 0)
#2
data1.loc[data1['Подавленность'] < 3, 'Подавленность2'] = 0
data1.loc[(data1['Подавленность'] >= 3) & (data1['Подавленность'] < 12), 'Подавленность2'] = 1
data1.loc[data1['Подавленность'] >= 12, 'Подавленность2'] = 2

data1.loc[data1['Тревожность'] < 1, 'Тревожность2'] = 0
data1.loc[(data1['Тревожность'] >= 1) & (data1['Тревожность'] < 8), 'Тревожность2'] = 1
data1.loc[data1['Тревожность'] >= 8, 'Тревожность2'] = 2

data1.loc[data1['Стресс'] < 5, 'Стресс2'] = 0
data1.loc[(data1['Стресс'] >= 5) & (data1['Стресс'] < 13), 'Стресс2'] = 1
data1.loc[data1['Стресс'] >= 13, 'Стресс2'] = 2
#3
data1['Подавленность3'] = data1['Подавленность'].apply(lambda x: 1 if x > 12 else 0)
data1['Тревожность3'] = data1['Тревожность'].apply(lambda x: 1 if x > 8 else 0)
data1['Стресс3'] = data1['Стресс'].apply(lambda x: 1 if x > 13 else 0)

In [154]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score

target_var_list_needed = ['Подавленность1', 'Подавленность2', 'Подавленность3', 
                   'Тревожность1', 'Тревожность2', 'Тревожность3', 
                   'Стресс1', 'Стресс2', 'Стресс3']


target_var_list = ['Подавленность1', 'Подавленность3', 
                   'Тревожность1', 'Тревожность3', 
                   'Стресс1', 'Стресс3']

X = data1[['sex', 'status', 'albums_count', 'photos_count', 'friends_count',
       'groups_count', 'followers_count_y', 'subscriptions_count',
       'Подавленность', 'Тревожность', 'Стресс', 'age', 'university', 'school',
       'career', 'profile_fulness', 'subdivision_1', 'subdivision_2',
       'subdivision_3', 'subdivision_4']]

result_matrix = []

for feature in target_var_list:
    y = data1[feature]

    X_train, X_testval, y_train, y_testval = train_test_split(X, y, test_size=0.2, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_testval, y_testval, test_size=0.5, random_state=10)

    parameters = {'loss':('hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'),
              'penalty':('l2', 'l1', 'elasticnet'),
              'alpha':[0.0001, 0.001, 0.01],
              'max_iter':[500, 1000, 1200]}
        
    #CrossVal
    clf = SGDClassifier()
    grid = GridSearchCV(clf, parameters, scoring = 'f1_weighted', cv = 5)
    grid = grid.fit(X_val, y_val)

    #Model
    model = SGDClassifier(**grid.best_params_)
    fit_model = model.fit(X_train, y_train)
    y_pred = fit_model.predict(X_test)
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
        
    result = [f1, precision, recall]
        
    result_matrix.append(result)

c:\users\svebb\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:570: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
c:\users\svebb\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:570: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
c:\users\svebb\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:570: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
c:\users\svebb\appdata\local\programs\python\python38-32\lib\sit

In [155]:
result_matrix

[[0.8398576512455517, 0.7814569536423841, 0.9076923076923077],
 [0.7049180327868853, 0.6323529411764706, 0.7962962962962963],
 [0.8189655172413793, 0.8260869565217391, 0.811965811965812],
 [0.6428571428571429, 0.5844155844155844, 0.7142857142857143],
 [0.8034934497816594, 0.7666666666666667, 0.8440366972477065],
 [0.6787878787878788, 0.5490196078431373, 0.8888888888888888]]